In [ ]:
! pip install torchtext==0.10.1
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
Looking in indexes: https://pypi.org/

In [ ]:
from datasets import load_dataset

import torch
from torchtext.legacy import data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PROJECT_ROOT = F"/content/gdrive/My Drive/nlp_project_task_2/"

In [ ]:
SEED = 42
MAX_VOCAB_SIZE = 25_000

In [ ]:
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
faithdial_dataset = load_dataset("McGill-NLP/FaithDial")

Extracting data files:   0%|          | 0/7 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/3539 [00:00<?, ? examples/s]

Generating test_random_split split:   0%|          | 0/1716 [00:00<?, ? examples/s]

Generating test_topic_split split:   0%|          | 0/1823 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/18357 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3417 [00:00<?, ? examples/s]

Generating valid_random_split split:   0%|          | 0/1666 [00:00<?, ? examples/s]

Generating valid_topic_split split:   0%|          | 0/1751 [00:00<?, ? examples/s]

Dataset faith_dial downloaded and prepared to /root/.cache/huggingface/datasets/McGill-NLP___faith_dial/plain_text/1.0.0/70568c8ab3bbc83b603bce58fa593ab27e7f0d0cde51034e1c2073ff3e14189a. Subsequent calls will reuse this data.


  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
faithdial_dataset.keys()

dict_keys(['test', 'test_random_split', 'test_topic_split', 'train', 'validation', 'valid_random_split', 'valid_topic_split'])

In [ ]:
faithdial_dataset["train"][0]

{'dialog_idx': 0,
 'response': 'Yeah, but once the access to the internet was a rare thing. do you remember?',
 'original_response': "No I could not! I couldn't imagine living when internet access was rare and very few people had it!",
 'history': ['Can you imagine the world without internet access?'],
 'knowledge': 'Internet access was once rare, but has grown rapidly.',
 'BEGIN': ['Hallucination'],
 'VRM': ['Disclosure', 'Ack.']}

In [ ]:
for d in faithdial_dataset["train"]:
    if d["original_response"] == None:
        print(d)
        break

{'dialog_idx': 2, 'response': 'woman were the main home brewers before beer was even commercialized', 'original_response': None, 'history': ['I have homebrewed before. Beer can be fun to craft.', "That's interesting, so you just brew enough for yourself?", 'Yes, that is correct. And it can be used to make some really cool concoctions.', "That's cool! Did you know that beer has been made by home brewers long before commercial companies made it?", 'Home brewers have been around for centuries if not thousands of years.', 'Yes, and the first beers were thick! Have you ever made a mead or a rice wine?', 'I have made mead, but that info about rice wine is interesting. Rice is also very versatile. What else do you know about home brewing?'], 'knowledge': 'Women brewers dominated alcohol production on every occupied continent until commercialization and industrialization of brewing occurred.', 'BEGIN': ['Entailment'], 'VRM': ['Edification']}


In [ ]:
def critic_preprocess(dataset):
    """
    Data items transformed into (knowledge, response, is_hallucination)
    """

    # BEGIN [('Hallucination', 13507), ('Entailment', 12663), ('Uncooperative', 979), ('Generic', 94)]
    # VRM [('Disclosure', 10101), ('Edification', 7592), ('Ack.', 6392), ('Question', 1590), ('Advisement', 562), ('', 49), (' Advisement', 1), (' Edification', 1)]

    new_dataset = []
    for d in dataset:
        # original response
        if d["original_response"] != None:
            new_dataset.append({
                "knowledge": d["knowledge"],
                "response": d["original_response"],
                "hallucination": "yes" if "Hallucination" in d["BEGIN"] else "no",
                "entailment": "yes" if "Entailment" in d["BEGIN"] else "no",
                "uncooperative": "yes" if "Uncooperative" in d["BEGIN"] else "no",
                "generic": "yes" if "Generic" in d["BEGIN"] else "no",
                "disclosure": "yes" if "Disclosure" in d["VRM"] else "no",
                "edification": "yes" if "Edification" in d["VRM"] else "no",
                "ack": "yes" if "Ack." in d["VRM"] else "no",
                "question": "yes" if "Question" in d["VRM"] else "no",
                "advisement": "yes" if "Advisement" in d["VRM"] else "no",
                "history": " \\ ".join(d["history"])
            })

        # new responses always aren't hallucinations
        # new_dataset.append({"knowledge": d["knowledge"],
        #                     "response": d["response"],
        #                     "BEGIN": "Entailment",
        #                     "VRM": d["VRM"],
        #                     "history": "\\".join(d["history"])
        # })
    return new_dataset

In [ ]:
import json

def dump_as_json(dataset, filename):
    """
    Takes a list of dicts and dumps it as a json file that torchtext can parse.
    """
    with open(filename, "w") as file:
        for d in dataset:
            file.write(json.dumps(d))
            file.write("\n")


In [ ]:
KNOWLEDGE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
RESPONSE = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)
HISTORY = data.Field(tokenize='spacy', tokenizer_language="en_core_web_sm", include_lengths = True)

# BEGIN [('Hallucination', 13507), ('Entailment', 12663), ('Uncooperative', 979), ('Generic', 94)]
HALLUCINATION = data.LabelField(dtype=torch.float)
ENTAILMENT = data.LabelField(dtype=torch.float)
UNCOOPERATIVE = data.LabelField(dtype=torch.float)
GENERIC = data.LabelField(dtype=torch.float)

# VRM [('Disclosure', 10101), ('Edification', 7592), ('Ack.', 6392), ('Question', 1590), ('Advisement', 562), ('', 49), (' Advisement', 1), (' Edification', 1)]
DISCLOSURE = data.LabelField(dtype=torch.float)
EDIFICATION = data.LabelField(dtype=torch.float)
ACK = data.LabelField(dtype=torch.float)
QUESTION = data.LabelField(dtype=torch.float)
ADVISEMENT = data.LabelField(dtype=torch.float)

In [ ]:
dump_as_json(critic_preprocess(faithdial_dataset["test"]), PROJECT_ROOT + "data/faithdial_dataset_test.json")
dump_as_json(critic_preprocess(faithdial_dataset["train"]), PROJECT_ROOT + "data/faithdial_dataset_train.json")
dump_as_json(critic_preprocess(faithdial_dataset["validation"]), PROJECT_ROOT + "data/faithdial_dataset_validation.json")

In [ ]:
fields = {
    "knowledge": ("k", KNOWLEDGE),
    "response": ("r", RESPONSE),
    "hallucination": ("bh", HALLUCINATION),
    "entailment": ("be", ENTAILMENT),
    "uncooperative": ("bu", UNCOOPERATIVE),
    "generic": ("bg", GENERIC),
    "disclosure": ("vd", DISCLOSURE),
    "edification": ("ve", EDIFICATION),
    "ack": ("vac", ACK),
    "question": ("vq", QUESTION),
    "advisement": ("vad", ADVISEMENT),
    "history": ("h", HISTORY)
}

dataset = data.TabularDataset.splits(path=PROJECT_ROOT + "data",
                                     train="faithdial_dataset_train.json",
                                     validation="faithdial_dataset_validation.json",
                                     test="faithdial_dataset_test.json",
                                     format="json",
                                     fields=fields)


In [ ]:
train_data, valid_data, test_data = dataset

In [ ]:
train_data[0]

In [ ]:
vars(train_data.examples[0])

{'k': ['Internet',
  'access',
  'was',
  'once',
  'rare',
  ',',
  'but',
  'has',
  'grown',
  'rapidly',
  '.'],
 'r': ['No',
  'I',
  'could',
  'not',
  '!',
  'I',
  'could',
  "n't",
  'imagine',
  'living',
  'when',
  'internet',
  'access',
  'was',
  'rare',
  'and',
  'very',
  'few',
  'people',
  'had',
  'it',
  '!'],
 'bh': 'yes',
 'be': 'no',
 'bu': 'no',
 'bg': 'no',
 'vd': 'yes',
 've': 'no',
 'vac': 'yes',
 'vq': 'no',
 'vad': 'no',
 'h': ['Can',
  'you',
  'imagine',
  'the',
  'world',
  'without',
  'internet',
  'access',
  '?']}

In [ ]:
KNOWLEDGE.build_vocab(train_data,
                      max_size=MAX_VOCAB_SIZE,
                      vectors = "fasttext.simple.300d",
                      unk_init = torch.Tensor.normal_)
RESPONSE.build_vocab(train_data,
                     max_size=MAX_VOCAB_SIZE,
                     vectors = "fasttext.simple.300d",
                     unk_init = torch.Tensor.normal_)
HISTORY.build_vocab(train_data,
                     max_size=MAX_VOCAB_SIZE,
                     vectors = "fasttext.simple.300d",
                     unk_init = torch.Tensor.normal_)

HALLUCINATION.build_vocab(train_data)
ENTAILMENT.build_vocab(train_data)
UNCOOPERATIVE.build_vocab(train_data)
GENERIC.build_vocab(train_data)

DISCLOSURE.build_vocab(train_data)
EDIFICATION.build_vocab(train_data)
ACK.build_vocab(train_data)
QUESTION.build_vocab(train_data)
ADVISEMENT.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in KNOWLEDGE vocabulary: {len(KNOWLEDGE.vocab)}")
print(f"Unique tokens in RESPONSE vocabulary: {len(RESPONSE.vocab)}")
print(f"Unique tokens in HISTORY vocabulary: {len(HISTORY.vocab)}")
print(f"Unique tokens in BEGIN vocabulary: {len(HALLUCINATION.vocab)}")
print(f"Unique tokens in VRM vocabulary: {len(DISCLOSURE.vocab)}")

Unique tokens in KNOWLEDGE vocabulary: 23219
Unique tokens in RESPONSE vocabulary: 21350
Unique tokens in HISTORY vocabulary: 21625
Unique tokens in BEGIN vocabulary: 2
Unique tokens in VRM vocabulary: 2


In [ ]:
print(KNOWLEDGE.vocab.freqs.most_common(20))
print(RESPONSE.vocab.freqs.most_common(20))
print(HISTORY.vocab.freqs.most_common(20))
print(HALLUCINATION.vocab.freqs.most_common(20))
print(DISCLOSURE.vocab.freqs.most_common(20))

[(',', 28389), ('the', 17584), ('.', 15545), ('and', 13312), ('of', 12349), ('a', 9358), ('in', 8886), ('is', 7942), ("''", 7533), ('to', 5682), ('or', 4583), ('as', 4422), (')', 3918), ('-', 3842), ('(', 3753), ('The', 3601), ('by', 3053), ('with', 2645), ('for', 2613), ('are', 2336)]
[('.', 15950), (',', 12171), ('the', 11405), ('I', 8748), ('a', 7375), ('and', 6829), ('of', 6616), ('is', 6002), ('to', 5873), ('in', 5562), ('it', 4572), ('!', 4416), ('that', 3600), ('are', 3194), ("'s", 2907), ('you', 2854), (' ', 2564), ('have', 2508), ('was', 2195), ('for', 2190)]
[('.', 59004), ('\\', 55153), ('I', 52661), (',', 51325), ('the', 33842), ('?', 32510), ('a', 27194), ('you', 24778), ('to', 21165), ('of', 20710), ('that', 20040), ('is', 19675), ('know', 17288), ('and', 17154), ('it', 16584), ('in', 15039), ('do', 12561), ('!', 11120), ('have', 10992), ("'s", 10708)]
[('yes', 13507), ('no', 2117)]
[('yes', 10101), ('no', 5523)]


In [ ]:
print(KNOWLEDGE.vocab.itos[:10])
print(RESPONSE.vocab.itos[:10])
print(HISTORY.vocab.itos[:10])
print(HALLUCINATION.vocab.itos[:10])
print(DISCLOSURE.vocab.itos[:10])

['<unk>', '<pad>', ',', 'the', '.', 'and', 'of', 'a', 'in', 'is']
['<unk>', '<pad>', '.', ',', 'the', 'I', 'a', 'and', 'of', 'is']
['<unk>', '<pad>', '.', '\\', 'I', ',', 'the', '?', 'a', 'you']
['yes', 'no']
['yes', 'no']


In [ ]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x: x.r,
    device = device)

In [ ]:
for batch in train_iterator:
    print(batch.bh)
    print(batch.vd)
    print(torch.stack((batch.bh, batch.vd), axis=1))
    break

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 0., 0.], device='cuda:0')
tensor([1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 0., 1., 0.], device='cuda:0')
tensor([[0., 1.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0

In [ ]:
from torch import nn

class LSTM(nn.Module):
    def __init__(self, response_vocab_size, knowledge_vocab_size, history_vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout, response_pad_idx, knowledge_pad_idx, history_pad_idx):

        super().__init__()

        # Initialize Embedding Layer
        self.response_embedding = nn.Embedding(num_embeddings=response_vocab_size,
                                               embedding_dim=embedding_dim,
                                               padding_idx=response_pad_idx)

        self.knowledge_embedding = nn.Embedding(num_embeddings=knowledge_vocab_size,
                                                embedding_dim=embedding_dim,
                                                padding_idx=knowledge_pad_idx)
        
        self.history_embedding = nn.Embedding(num_embeddings=history_vocab_size,
                                              embedding_dim=embedding_dim,
                                              padding_idx=history_pad_idx)

        # Initialize LSTM layer
        self.response_lstm = nn.LSTM(input_size=embedding_dim,
                                     hidden_size=hidden_dim,
                                     num_layers=n_layers,
                                     bidirectional=bidirectional)

        self.knowledge_lstm = nn.LSTM(input_size=embedding_dim,
                                      hidden_size=hidden_dim,
                                      num_layers=n_layers,
                                      bidirectional=bidirectional)
        
        self.history_lstm = nn.LSTM(input_size=embedding_dim,
                                    hidden_size=hidden_dim,
                                    num_layers=n_layers,
                                    bidirectional=bidirectional)

        # Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(in_features=3*2*hidden_dim,
                            out_features=output_dim)

        # Initialize Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, response, response_lengths, knowledge, knowledge_lengths, history, history_lengths):
        # Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        x_r = self.response_embedding(response)
        x_r = self.dropout(x_r)

        x_k = self.knowledge_embedding(knowledge)
        x_k = self.dropout(x_k)

        x_h = self.history_embedding(history)
        x_h = self.dropout(x_h)

        # Run the LSTM along the sentences of length sent_len.
        output_r, (hidden_r, cell_r) = self.response_lstm(x_r)
        output_k, (hidden_k, cell_k) = self.knowledge_lstm(x_k)
        output_h, (hidden_h, cell_h) = self.history_lstm(x_h)

        # Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        hidden_r = torch.cat((hidden_r[-2,:,:], hidden_r[-1,:,:]), -1)
        hidden_k = torch.cat((hidden_k[-2,:,:], hidden_k[-1,:,:]), -1)
        hidden_h = torch.cat((hidden_h[-2,:,:], hidden_h[-1,:,:]), -1)
        hidden = torch.cat((hidden_r, hidden_k, hidden_h), -1)
        hidden = self.dropout(hidden)

        return self.fc(hidden)

In [ ]:
RESPONSE_INPUT_DIM = len(RESPONSE.vocab)
KNOWLEDGE_INPUT_DIM = len(KNOWLEDGE.vocab)
HISTORY_INPUT_DIM = len(HISTORY.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
# OUTPUT_DIM = 1
OUTPUT_DIM = 9
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
RESPONSE_PAD_IDX = RESPONSE.vocab.stoi[RESPONSE.pad_token]
KNOWLEDGE_PAD_IDX = KNOWLEDGE.vocab.stoi[KNOWLEDGE.pad_token]
HISTORY_PAD_IDX = HISTORY.vocab.stoi[HISTORY.pad_token]

model = LSTM(RESPONSE_INPUT_DIM,
             KNOWLEDGE_INPUT_DIM,
             HISTORY_INPUT_DIM,
             EMBEDDING_DIM,
             HIDDEN_DIM,
             OUTPUT_DIM,
             N_LAYERS,
             BIDIRECTIONAL,
             DROPOUT,
             RESPONSE_PAD_IDX,
             KNOWLEDGE_PAD_IDX,
             HISTORY_PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 28,031,265 trainable parameters


In [ ]:
print(RESPONSE.vocab.vectors.shape)
print(KNOWLEDGE.vocab.vectors.shape)
print(HISTORY.vocab.vectors.shape)

torch.Size([21350, 300])
torch.Size([23219, 300])
torch.Size([21625, 300])


In [ ]:
model.response_embedding.weight.data.copy_(RESPONSE.vocab.vectors)
model.knowledge_embedding.weight.data.copy_(KNOWLEDGE.vocab.vectors)
model.history_embedding.weight.data.copy_(HISTORY.vocab.vectors)

tensor([[-0.0320, -0.6273,  0.3034,  ..., -1.2316,  0.3065, -0.4413],
        [-0.0486, -0.5080, -0.7784,  ...,  0.0578,  1.8190, -1.3260],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [ 0.3162,  0.8552,  0.0060,  ...,  0.5351, -1.2282, -0.1591],
        [-0.4821,  1.4751, -0.9519,  ..., -1.5179, -0.5748,  0.6027],
        [-1.0237,  0.0260, -1.7186,  ...,  0.1113,  0.1900, -1.8552]])

In [ ]:
UNK_IDX_R = RESPONSE.vocab.stoi[RESPONSE.unk_token]
UNK_IDX_K = RESPONSE.vocab.stoi[KNOWLEDGE.unk_token]
UNK_IDX_H = RESPONSE.vocab.stoi[HISTORY.unk_token]

model.response_embedding.weight.data[UNK_IDX_R] = torch.zeros(EMBEDDING_DIM)
model.response_embedding.weight.data[RESPONSE_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

model.knowledge_embedding.weight.data[UNK_IDX_K] = torch.zeros(EMBEDDING_DIM)
model.knowledge_embedding.weight.data[KNOWLEDGE_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

model.history_embedding.weight.data[UNK_IDX_H] = torch.zeros(EMBEDDING_DIM)
model.history_embedding.weight.data[HISTORY_PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.response_embedding.weight.data)
print(model.knowledge_embedding.weight.data)
print(model.history_embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0569, -0.0520,  0.2733,  ..., -0.0695, -0.1606, -0.0989],
        ...,
        [-1.2877,  0.9261,  0.0691,  ...,  0.3227, -1.0139, -1.8438],
        [ 1.1394, -0.6671,  0.0390,  ..., -0.3197,  1.1253, -1.6944],
        [-1.3534, -0.4667, -0.3012,  ..., -0.6564,  0.0746, -1.3957]])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [-0.2951, -0.8804, -0.1080,  ..., -0.3395,  1.1827, -0.2405],
        [ 1.1217,  0.4453, -0.9606,  ..., -0.7635,  0.3417, -0.6243],
        [ 0.9979,  0.2376,  0.0183,  ...,  1.4123,  0.1321, -0.8839]])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
fields

{'knowledge': ('k', <torchtext.legacy.data.field.Field at 0x7f281f2d1df0>),
 'response': ('r', <torchtext.legacy.data.field.Field at 0x7f29584b27f0>),
 'hallucination': ('bh',
  <torchtext.legacy.data.field.LabelField at 0x7f29584b28b0>),
 'entailment': ('be',
  <torchtext.legacy.data.field.LabelField at 0x7f281ee11cd0>),
 'uncooperative': ('bu',
  <torchtext.legacy.data.field.LabelField at 0x7f29584aca00>),
 'generic': ('bg', <torchtext.legacy.data.field.LabelField at 0x7f281e9808b0>),
 'disclosure': ('vd',
  <torchtext.legacy.data.field.LabelField at 0x7f28507fcc40>),
 'edification': ('ve',
  <torchtext.legacy.data.field.LabelField at 0x7f27fa955b20>),
 'ack': ('vac', <torchtext.legacy.data.field.LabelField at 0x7f281f2d1eb0>),
 'question': ('vq',
  <torchtext.legacy.data.field.LabelField at 0x7f27f247fd00>),
 'advisement': ('vad',
  <torchtext.legacy.data.field.LabelField at 0x7f27f2699e20>),
 'history': ('h', <torchtext.legacy.data.field.Field at 0x7f29dc20f970>)}

In [ ]:
def batch2label(b):
    return torch.stack((b.bh, b.be, b.bu, b.bg, b.vd, b.ve, b.vac, b.vq, b.vad), axis=1)

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / (correct.size()[0] * correct.size()[1])
    return acc


def binary_f1(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = torch.cat(rounded_preds.unbind(0), -1)
    y = torch.cat(y.unbind(0), -1)
    f1 = f1_score(y.detach().cpu(), rounded_preds.detach().cpu(), average="macro")

    return f1

def binary_confusion(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = torch.cat(rounded_preds.unbind(0), -1)
    y = torch.cat(y.unbind(0), -1)
    conf = confusion_matrix(y.detach().cpu(), rounded_preds.detach().cpu())

    return conf

In [ ]:
for batch in train_iterator:
    response, response_lengths = batch.r
    knowledge, knowledge_lengths = batch.k
    history, history_lengths = batch.h

    predictions = model(response, response_lengths, knowledge, knowledge_lengths, history, history_lengths).squeeze(1)
    label = batch2label(batch)
    #print(predictions)
    #print(label)
    # print(len(predictions))
    # print(len(label))
    # print(predictions.size())
    # print(predictions.sum())
    # print(predictions.size()[0] * predictions.size()[1])
    print(binary_accuracy(predictions, label))
    print(binary_f1(predictions, label))
    # print(label)
    # print(torch.cat(label.unbind(0), -1))
    conf = binary_confusion(predictions, label)
    print(conf)
    print(type(conf))
    break

tensor(0.9288, device='cuda:0')
0.9044254234201421
[[413  18]
 [ 23 122]]
<class 'numpy.ndarray'>


In [ ]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        response, response_lengths = batch.r
        knowledge, knowledge_lengths = batch.k
        history, history_lengths = batch.h

        predictions = model(response, response_lengths, knowledge, knowledge_lengths, history, history_lengths).squeeze(1)

        label = batch2label(batch)
        loss = criterion(predictions, label)

        acc = binary_accuracy(predictions, label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import numpy as np

def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0
    epoch_f1 = 0
    epoch_conf = np.array([[0, 0], [0, 0]])

    model.eval()

    with torch.no_grad():

        for batch in iterator:
            response, response_lengths = batch.r
            knowledge, knowledge_lengths = batch.k
            history, history_lengths = batch.h

            predictions = model(response, response_lengths, knowledge, knowledge_lengths, history, history_lengths).squeeze(1)

            label = batch2label(batch)
            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)
            f1 = binary_f1(predictions, label)
            conf = binary_confusion(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            epoch_f1 += f1.item()
            epoch_conf += conf

    return epoch_loss / len(iterator), epoch_acc / len(iterator), epoch_f1 / len(iterator), epoch_conf

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
# path = F"/content/gdrive/My Drive/bilstm_model.pt"
path = PROJECT_ROOT + F"/bilstm_model.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, valid_f1, _ = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% |')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% | Val. F1: {valid_f1:.3f}')

Epoch: 01 | Epoch Time: 0m 38s
	Train Loss: 0.233 | Train Acc: 90.06% |
	 Val. Loss: 0.285 |  Val. Acc: 87.78% | Val. F1: 0.781
Epoch: 02 | Epoch Time: 0m 39s
	Train Loss: 0.215 | Train Acc: 90.91% |
	 Val. Loss: 0.292 |  Val. Acc: 87.80% | Val. F1: 0.789
Epoch: 03 | Epoch Time: 0m 40s
	Train Loss: 0.196 | Train Acc: 91.81% |
	 Val. Loss: 0.310 |  Val. Acc: 87.50% | Val. F1: 0.788
Epoch: 04 | Epoch Time: 0m 41s
	Train Loss: 0.176 | Train Acc: 92.72% |
	 Val. Loss: 0.323 |  Val. Acc: 87.43% | Val. F1: 0.781
Epoch: 05 | Epoch Time: 0m 40s
	Train Loss: 0.159 | Train Acc: 93.44% |
	 Val. Loss: 0.350 |  Val. Acc: 87.36% | Val. F1: 0.780


In [ ]:
model.load_state_dict(torch.load(path, map_location=device))

test_loss, test_acc, test_f1, test_conf = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% | Test F1: {test_f1:.2f}')

Test Loss: 0.283 | Test Acc: 87.77% | Test F1: 0.78


In [ ]:
test_conf

array([[19752,  1036],
       [ 2186,  3342]])

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_hallucination(model, knowledge, response):
    model.eval()

    tokenized_r = [tok.text for tok in nlp.tokenizer(response)]
    indexed_r = [RESPONSE.vocab.stoi[t] for t in tokenized_r]
    length_r = [len(indexed_r)]
    tensor_r = torch.LongTensor(indexed_r).to(device)
    tensor_r = tensor_r.unsqueeze(1)
    length_tensor_r = torch.LongTensor(length_r)

    tokenized_k = [tok.text for tok in nlp.tokenizer(knowledge)]
    indexed_k = [KNOWLEDGE.vocab.stoi[t] for t in tokenized_k]
    length_k = [len(indexed_k)]
    tensor_k = torch.LongTensor(indexed_k).to(device)
    tensor_k = tensor_k.unsqueeze(1)
    length_tensor_k = torch.LongTensor(length_k)

    prediction = torch.sigmoid(model(tensor_r, length_tensor_r, tensor_k, length_tensor_k))

    return prediction.item()


In [ ]:
predict_hallucination(model, "", "I love dogs")

TypeError: ignored

In [ ]:
predict_hallucination(model, "", "Dogs are animals.")

In [ ]:
predict_hallucination(model, "", "I was walking my dog last week.")

In [ ]:
predict_hallucination(model, "", "Dogs need to be walked daily.")

In [ ]:
test_data[2].r

In [ ]:
predict_hallucination(model, "", "Dylan's Candy Bar is a candy supplier.")

In [ ]:
predict_hallucination(model, "", "Dylan's Candy Bar is my favorite great brand of candy.")

In [ ]:
print(test_data[2].h)

['I', 'love', 'candy', ',', 'what', "'s", 'a', 'good', 'brand?\\I', 'do', "n't", 'know', 'how', 'good', 'they', 'are', ',', 'but', 'Dylan', "'s", 'Candy', 'Bar', 'has', 'a', 'chain', 'of', 'candy', 'shops', 'in', 'various', 'cities.\\Oh', ',', 'they', 'do', '?', 'What', 'kind', 'of', 'candy', 'do', 'they', 'sell', '?']
